# Fit `polyclonal` model for antibody.

Import Python modules.

In [1]:
import altair as alt

import polyclonal

import pandas as pd

This notebook is parameterized by `papermill`.
The next cell is tagged as `parameters` to get the passed parameters.

In [2]:
# this cell is tagged parameters for `papermill` parameterization
selection = None
params = None

In [3]:
# Parameters
params = {
    "antibody_samples": {
        "LibA-220210-REGN10933-0.15-1": {"concentration": 0.15, "use_in_fit": True},
        "LibA-220210-REGN10933-1.39-1": {"concentration": 1.39, "use_in_fit": True},
        "LibA-220210-REGN10933-5.58-1": {"concentration": 5.58, "use_in_fit": True},
    },
    "neut_standard_name": "neut_standard",
    "no_antibody_sample": "LibA-220210-no_antibody-1",
    "prob_escape_filters": {
        "min_antibody_counts": 10,
        "min_antibody_frac": 0.005,
        "min_neut_standard_count": 1000,
        "min_neut_standard_frac": 0.001,
        "min_no_antibody_counts": 5,
        "min_no_antibody_frac": "5e-05",
    },
}
selection = "LibA-220210-REGN10933-1"

import os
os.chdir("../test_example")

## Read and process data
Convert the antibody samples into a data frame:

In [4]:
antibody_samples = (
    pd.DataFrame.from_dict(params["antibody_samples"], orient="index")
    .reset_index(names="sample")
)

Get other parameters:

In [5]:
prob_escape_filters = params["prob_escape_filters"]

Read the neut standard fracs:

In [6]:
neut_standard_fracs = pd.concat(
    [
        pd.read_csv(
            f"results/antibody_escape/{selection}/{sample}_neut_standard_fracs.csv"
        ).assign(sample=sample)
        for sample in antibody_samples["sample"]
    ],
    ignore_index=True,
).merge(antibody_samples, validate="one_to_one", on="sample")

Read the probabilities (fraction) escape for each variant:

In [7]:
prob_escape = pd.concat(
    [
        pd.read_csv(
            f"results/antibody_escape/{selection}/{sample}_prob_escape.csv",
            keep_default_na=False,
            na_values="nan",
        ).assign(sample=sample)
        for sample in antibody_samples["sample"]
    ],
    ignore_index=True,
).merge(antibody_samples, validate="many_to_one", on="sample")

## Plot the neutralization standard fractions

Plot the neutralization standard fractions for each sample:

In [8]:
neut_standard_fracs_chart = (
    alt.Chart(
        neut_standard_fracs
        .rename(columns={"antibody_frac": "antibody", "no-antibody_frac": "no-antibody"})
        .melt(
            id_vars=["sample", "use_in_fit", "concentration"],
            value_vars=["antibody", "no-antibody"],
            var_name="sample type",
            value_name="neutralization standard fraction",
        )
        .assign(use_in_fit=lambda x: x["use_in_fit"].where(x["concentration"] > 1, False))
    )
    .encode(
        x=alt.X(
            "neutralization standard fraction",
            scale=alt.Scale(type="symlog", constant=0.04, domainMax=1),
        ),
        y=alt.Y("sample", sort=alt.SortField("concentration"), title=None),
        shape=alt.Shape("sample type", title="sample type (filled if used in fit)"),
        stroke=alt.Color("sample type", scale=alt.Scale(range=["#1F77B4FF", "#FF7F0EFF"])),
        color=alt.Color("sample type", scale=alt.Scale(range=["#1F77B4FF", "#FF7F0EFF"])),
        fillOpacity=alt.Opacity(
            "use_in_fit",
            scale=alt.Scale(domain=[True, False], range=[1, 0]),
        ),
        tooltip=[
            "sample",
            alt.Tooltip("concentration", format=".3g"),
            alt.Tooltip("neutralization standard fraction", format=".3g"),
        ],
    )
    .mark_point(filled=True, size=50)
    .configure_axis(labelLimit=500)
    .properties(title=f"Neutralization standard fractions for {selection}")
)

neut_standard_fracs_chart

alt.Chart(...)

Make sure all samples used in the fit have enough neutralization standard counts and fraction:

In [9]:
for prop in ["count", "frac"]:
    minval = float(prob_escape_filters[f"min_neut_standard_{prop}"])
    minval = float(minval)
    if all(
        (neut_standard_fracs.query("use_in_fit")[f"{stype}_{prop}"] >= minval).all()
        for stype in ["antibody", "no-antibody"]
    ):
        print(f"Adequate neut_standard_{prop} of >= {minval}")
    else:
        raise ValueError(f"Inadequate neut_standard_{prop} < {minval}\n{neut_standard_fracs}")

Adequate neut_standard_count of >= 1000.0
Adequate neut_standard_frac of >= 0.001


In [10]:
all

<function all(iterable, /)>